In [0]:
# =========================================================
# GOLD - Campaign Targets (Strategy Mapping Engine)
# =========================================================
# 역할:
#   - 모델 예측 결과(gold_churn_predictions) + 행동 스냅샷 기반
#   - 최종 마케팅 전략 확정(전략코드/우선순위/발송여부)
#
# 전략 우선순위(priority_rank) — 문서 기준 고정:
#   1 Save   : 해지 직전 고위험 방어
#   2 Start  : 초기 온보딩 실패 복구
#   3 Fix    : 기술적 단절 해결
#   4 React  : 고위험 이용 감소 재참여
#   5 Finish : 몰입 후 급감 사용자 재개 유도
#   6 Boost  : 고활동 유저 몰입 강화
#   7 Stay   : 완만한 감소 유지
#   8 Calm   : 발송 억제
#
# Activity Pattern 정의표(핵심 조건):
#   Save   : probability_band=Critical AND days_since_last_login>=14
#   Start  : observation_days<=30 AND active_days_30<=2
#   Fix    : frequency_active_days<=2 AND observation_days>=30
#   React  : watch_time_30d_min>=120 AND watch_time_7d_min < watch_time_30d_min*0.3
#   Finish : watch_time_30d_min>=300 AND watch_time_7d_min<30 AND active_days_30>=5
#   Boost  : segment=Heavy AND probability_band=High
#   Stay   : probability_band=Mid AND watch_time_7d_min < watch_time_30d_min/4
#   Calm   : probability_band=Low
#
# ⚠ RAG 연동:
#   - 이 테이블은 RAG 입력 SSOT입니다.
#   - send_flag=1 대상만 메시지 생성
#   - RAG 결과 로그는 Gold를 늘리지 않기 위해 Silver에 저장 권장
# =========================================================

from pyspark.sql import functions as F

CATALOG = "signalcraft_databricks"
SCHEMA  = "default"

PRED_TABLE = f"{CATALOG}.{SCHEMA}.gold_churn_predictions"          # ✅ 예측 결과(SSOT)
SNAP_TABLE = f"{CATALOG}.{SCHEMA}.dlt_gold_user_behavior_snapshot" # ✅ 스냅샷
OUT_TABLE  = f"{CATALOG}.{SCHEMA}.gold_campaign_targets"           # ✅ 최종 타겟

# -----------------------------
# 1) score_date 결정 (T-1 기본, 백필 파라미터 optional)
# -----------------------------
dbutils.widgets.text("score_date", "")
param_date = dbutils.widgets.get("score_date").strip()

t1 = spark.sql("SELECT date_sub(current_date(), 1) AS d").first()["d"]
score_date = param_date if param_date else t1

print(f"📅 targets 기준일(score_date): {score_date}")

# -----------------------------
# 2) 준비 체크 (예측이 먼저 있어야 함)
# -----------------------------
pred_cnt = (spark.table(PRED_TABLE)
            .filter(F.col("event_date") == F.lit(score_date))
            .count())
if pred_cnt == 0:
    raise RuntimeError(f"[STOP] gold_churn_predictions not ready for event_date={score_date}")

snap_cnt = (spark.table(SNAP_TABLE)
            .filter(F.col("event_date") == F.lit(score_date))
            .count())
if snap_cnt == 0:
    raise RuntimeError(f"[STOP] snapshot not ready for event_date={score_date}")

print(f"✅ pred rows: {pred_cnt}, snap rows: {snap_cnt}")

# -----------------------------
# 3) pred + snapshot 조인
# -----------------------------
pred = (spark.table(PRED_TABLE)
        .filter(F.col("event_date") == F.lit(score_date))
        .select(
            "event_date",
            "user_id",
            F.col("probability_band").alias("probability_band"),
            F.col("churn_reason").alias("churn_reason"),
        ))

snap = (spark.table(SNAP_TABLE)
        .filter(F.col("event_date") == F.lit(score_date))
        .select(
            "event_date", "user_id",
            "days_since_last_login",
            "observation_days",
            "frequency_active_days",
            "watch_time_7d_min",
            "watch_time_30d_min",
            "active_days_30",
            "segment"
        ))

base = pred.alias("p").join(snap.alias("s"), ["event_date", "user_id"], "left")

# -----------------------------
# 4) Activity Pattern 조건
# -----------------------------
cond_save = (
        (F.col("p.probability_band") == "Critical") &
        (F.col("s.days_since_last_login") >= 7) #기존에는 14
    )

cond_start = (
    (F.col("s.observation_days") <= 30) &
    (F.col("s.active_days_30") <= 2)
)

cond_fix = (
    (F.col("s.frequency_active_days") <= 2) &
    (F.col("s.observation_days") >= 30)
)

cond_react = (
    (F.col("s.watch_time_30d_min") >= 120) &
    (F.col("s.watch_time_7d_min") < (F.col("s.watch_time_30d_min") * F.lit(0.3)))
)

cond_finish = (
    (F.col("s.watch_time_30d_min") >= 300) &
    (F.col("s.watch_time_7d_min") < 30) &
    (F.col("s.active_days_30") >= 5)
)

cond_boost = (
    (F.col("s.segment") == "Heavy") &
    (F.col("p.probability_band") == "High")
)

cond_stay = (
    (F.col("p.probability_band") == "Mid") &
    (F.col("s.watch_time_7d_min") < (F.col("s.watch_time_30d_min") / F.lit(4)))
)

cond_calm = (F.col("p.probability_band") == "Low")


# ---- 우선순위 표 그대로 적용(1~8) ----
strategy_code = (
    F.when(cond_save,   F.lit("Save"))
     .when(cond_start,  F.lit("Start"))
     .when(cond_fix,    F.lit("Fix"))
     .when(cond_react,  F.lit("React"))
     .when(cond_finish, F.lit("Finish"))
     .when(cond_boost,  F.lit("Boost"))
     .when(cond_stay,   F.lit("Stay"))
     .when(cond_calm,   F.lit("Calm"))
     .otherwise(F.lit("Calm"))
)

priority_rank = (
    F.when(strategy_code == "Save",   F.lit(1))
     .when(strategy_code == "Start",  F.lit(2))
     .when(strategy_code == "Fix",    F.lit(3))
     .when(strategy_code == "React",  F.lit(4))
     .when(strategy_code == "Finish", F.lit(5))
     .when(strategy_code == "Boost",  F.lit(6))
     .when(strategy_code == "Stay",   F.lit(7))
     .otherwise(F.lit(8))
)

# Calm은 발송 억제
send_flag = F.when(strategy_code == "Calm", F.lit(0)).otherwise(F.lit(1))

out_df = (base
          .withColumn("strategy_code", strategy_code)
          .withColumn("priority_rank", priority_rank.cast("int"))
          .withColumn("send_flag", send_flag.cast("int"))
          .select(
              "event_date", "user_id",
              "probability_band", "churn_reason",
              "strategy_code", "priority_rank", "send_flag"
          ))

# -----------------------------
# 5) 저장: 해당 날짜만 overwrite (히스토리 유지)
# -----------------------------
(out_df.write
 .format("delta")
 .mode("overwrite")
 .option("replaceWhere", f"event_date = '{score_date}'")
 .saveAsTable(OUT_TABLE))

print(f"✅ saved: {OUT_TABLE} for event_date={score_date}")

📅 targets 기준일(score_date): 2026-02-19
✅ pred rows: 2367, snap rows: 10000
✅ saved: signalcraft_databricks.default.gold_campaign_targets for event_date=2026-02-19
